In [1]:
import findspark
findspark.init('/usr/local/spark')
import pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('case study-ETL-Assessment').getOrCreate()

In [3]:
from pyspark.sql import Row

In [4]:
sc = spark.sparkContext

In [5]:
from pyspark.sql.types import *

### Loading the data of the files into PySparkSQL dataframes.
### Print the schema of each dataframe and show to first 10 rows of each dataframe.

In [6]:
data = sc.textFile("NYSE_daily.tsv")

In [7]:
data = data.map(lambda l: l.split("\t"))
data = data.map(lambda p: (p[0],p[1],p[2],p[3],p[4],p[5],p[6],p[7],p[8]))

In [8]:
schemaString = "exchange symbol stdate oprice hprice lprice cprice vol adjclprice"

In [9]:
ccolumns = [StructField(column_name, StringType(), True) for column_name in schemaString.split()]
schema = StructType(ccolumns)

In [10]:
daily_df = spark.createDataFrame(data,schema)

In [11]:
daily_df.printSchema()

root
 |-- exchange: string (nullable = true)
 |-- symbol: string (nullable = true)
 |-- stdate: string (nullable = true)
 |-- oprice: string (nullable = true)
 |-- hprice: string (nullable = true)
 |-- lprice: string (nullable = true)
 |-- cprice: string (nullable = true)
 |-- vol: string (nullable = true)
 |-- adjclprice: string (nullable = true)



In [12]:
user = sc.textFile("NYSE_dividends.tsv")

In [13]:
user = user.map(lambda l: l.split("\t"))
user = user.map(lambda p: (p[0],p[1],p[2],p[3]))

In [14]:
schemaString = "exchange symbol stdate dividends"

In [15]:
ccolumns = [StructField(column_name, StringType(), True) for column_name in schemaString.split()]
schema = StructType(ccolumns)

In [16]:
dividends_df = spark.createDataFrame(user,schema)

In [17]:
dividends_df.printSchema()

root
 |-- exchange: string (nullable = true)
 |-- symbol: string (nullable = true)
 |-- stdate: string (nullable = true)
 |-- dividends: string (nullable = true)



In [18]:
daily_df.show(10)

+--------+------+---------+------+------+------+------+-------+----------+
|exchange|symbol|   stdate|oprice|hprice|lprice|cprice|    vol|adjclprice|
+--------+------+---------+------+------+------+------+-------+----------+
|    NYSE|   JEF| 2/8/2010|  25.4| 25.49| 24.78| 24.82|1134300|     24.82|
|    NYSE|   JEF| 2/5/2010| 24.91| 25.19| 24.08| 25.01|1765200|     25.01|
|    NYSE|   JEF| 2/4/2010| 26.01|  26.2| 24.85| 24.85|1414400|     24.85|
|    NYSE|   JEF| 2/3/2010| 26.23| 26.76| 26.22| 26.29|1066000|     26.29|
|    NYSE|   JEF| 2/2/2010| 26.08| 26.86| 25.78| 26.46|1496400|     26.46|
|    NYSE|   JEF| 2/1/2010| 25.61| 26.11| 25.36| 26.11|2381800|     26.11|
|    NYSE|   JEF|1/29/2010| 26.57|  26.8| 25.41| 25.54|2010000|     25.54|
|    NYSE|   JEF|1/28/2010|  27.4|  27.4| 26.35| 26.36|1708100|     26.36|
|    NYSE|   JEF|1/27/2010| 26.44| 27.15| 26.42| 27.14|1929700|     27.14|
|    NYSE|   JEF|1/26/2010| 26.68| 26.99| 26.46|  26.5|1422100|      26.5|
+--------+------+--------

In [19]:
dividends_df.show(10)

+--------+------+----------+---------+
|exchange|symbol|    stdate|dividends|
+--------+------+----------+---------+
|    NYSE|   JAH|12/30/2009|    0.075|
|    NYSE|   JAH| 9/29/2009|    0.075|
|    NYSE|   JGT|12/11/2009|    0.377|
|    NYSE|   JGT| 9/11/2009|    0.377|
|    NYSE|   JGT| 6/11/2009|    0.377|
|    NYSE|   JGT| 3/11/2009|    0.377|
|    NYSE|   JGT|12/11/2008|    0.377|
|    NYSE|   JGT| 9/11/2008|    0.451|
|    NYSE|   JGT| 6/11/2008|    0.451|
|    NYSE|   JGT| 3/12/2008|    0.451|
+--------+------+----------+---------+
only showing top 10 rows



In [20]:
daily_df.createOrReplaceTempView("nysedaily")
dividends_df.createOrReplaceTempView("nysedividends")

### Q1. List the companies which have the stock close price more than or equal to 200 and stock volume more than or equal to 10 million.

In [21]:
query_1=spark.sql("select * from nysedaily where cprice>=200 and vol>=10000000 limit 10")
query_1.show()

+--------+------+----------+------+------+------+------+--------+----------+
|exchange|symbol|    stdate|oprice|hprice|lprice|cprice|     vol|adjclprice|
+--------+------+----------+------+------+------+------+--------+----------+
|    NYSE|  JNPR| 11/3/2000|   198|216.88|196.25|216.13|13424800|    216.13|
|    NYSE|  JNPR|10/19/2000|229.13|234.31|   220|232.58|11323800|    232.58|
|    NYSE|  JNPR|10/18/2000|219.38|   235| 212.5|213.88|15463100|    213.88|
|    NYSE|  JNPR|10/17/2000|241.75|241.81|   224|229.19|16734200|    229.19|
|    NYSE|  JNPR|10/16/2000|226.75| 244.5|   224|   243|17288400|       243|
|    NYSE|  JNPR|10/13/2000|201.75| 229.5|201.63| 228.5|19565000|     228.5|
|    NYSE|  JNPR|10/11/2000| 201.5|219.44|196.19|   206|16487000|       206|
|    NYSE|  JNPR|10/10/2000| 196.5|   211|   196|205.94|12586900|    205.94|
|    NYSE|  JNPR| 10/4/2000|200.75|211.63| 191.5|207.95|11819100|    207.95|
|    NYSE|  JNPR| 10/3/2000|212.63|   215|   198|201.44|13457300|    201.44|

### Q2. List the companies that have given dividends more than 50 times. The list should include the number of times they have given dividends.

In [22]:
query_2=spark.sql("select symbol,count(dividends) as Max_Dividends from nysedividends group by symbol having Max_Dividends>50 order by Max_Dividends limit 10")
query_2.show()

+------+-------------+
|symbol|Max_Dividends|
+------+-------------+
|   JOE|           51|
|   JQC|           55|
|   JFP|           58|
|   JPC|           60|
|   JRO|           63|
|   JFR|           68|
|   JEF|           72|
|   JWN|           81|
|   JHP|           85|
|   JHS|           88|
+------+-------------+



###  Q3. List the companies along with their close price, dividends and the date when they gave dividends of more than 0.01 when their daily close price was more than or equal to 100 sorted by dividends in ascending order.

In [23]:
query_3=spark.sql('select d.cprice,div.dividends,div.stdate from nysedaily d,nysedividends div where d.symbol=div.symbol and div.dividends>0.01 and d.cprice>=100 sort by div.dividends')
query_3.show()

+------+---------+----------+
|cprice|dividends|    stdate|
+------+---------+----------+
|   100|     0.07|11/12/2009|
|   100|     0.07| 8/13/2009|
|   100|     0.07| 5/13/2009|
|   100|     0.07| 2/12/2009|
|   100|     0.07|11/13/2008|
|   100|     0.07| 8/13/2008|
|   100|     0.07| 5/13/2008|
|   100|     0.07| 2/13/2008|
|   100|     0.07|11/13/2007|
|   100|     0.07| 8/13/2007|
|   100|     0.07| 5/11/2007|
|   100|     0.07| 2/13/2007|
|   100|     0.07|11/13/2006|
|   100|     0.07| 8/11/2006|
|   100|     0.07| 5/11/2006|
|101.12|     0.07|11/12/2009|
|101.12|     0.07| 8/13/2009|
|101.12|     0.07| 5/13/2009|
|101.12|     0.07| 2/12/2009|
|101.12|     0.07|11/13/2008|
+------+---------+----------+
only showing top 20 rows



### Q4. Save the above lists as comma separated files.

In [25]:
query_1.coalesce(1).write.format("csv").save('/home/hduser/Downloads/sharedfolder/Query1')

In [26]:
query_2.coalesce(1).write.format("csv").save('/home/hduser/Downloads/sharedfolder/Query2')

In [27]:
query_3.coalesce(1).write.format("csv").save('/home/hduser/Downloads/sharedfolder/Query3')